In [ ]:
"""
Use a file OSM to detect tags not considered in the
OSM2LULC procedure
"""

In [1]:
# Parameters

OSM_FILE    = r'D:\indev\OSM2LULC\id_not_used_tags\osm_15.xml'
OSM_TAG_MAP = {
    "DB"        : r'D:\xpto\gasp\gasp\osm2lulc\osmtolulc.sqlite',
    "OSM_FEAT"  : 'osm_features',
    "KEY_COL"   : "key",
    "VALUE_COL" : "value",
    "GEOM_COL"  : "geom"
}

WORKSPACE   = r'D:\indev\OSM2LULC\id_not_used_tags'

In [2]:
import os
from gasp.oss import get_filename
from gasp.osm2lulc.utils import osm_to_sqdb
from gasp.fm.sqLite import sqlq_to_df
from gasp.cpu.gdl.anls.exct import sel_by_attr

In [3]:
#sqdb = osm_to_sqdb(OSM_FILE, os.path.join(WORKSPACE, get_filename(OSM_FILE) + '.sqlite'))
sqdb = r'D:\indev\OSM2LULC\id_not_used_tags\osm_15.sqlite'

In [4]:
# Get Features we are considering

ourOSMFeatures = sqlq_to_df(OSM_TAG_MAP["DB"], (
    "SELECT {key} AS key_y, {value} AS value_y, {geom} AS geom_y "
    "FROM {tbl}"
).format(
    key=OSM_TAG_MAP["KEY_COL"], value=OSM_TAG_MAP["VALUE_COL"],
    geom=OSM_TAG_MAP["GEOM_COL"], tbl=OSM_TAG_MAP["OSM_FEAT"]
))

In [5]:
# Get Features in File

TABLES_TAGS = {
    'points'        : ['highway', 'man_made', 'building'],
    'lines'         : ['highway', 'waterway', 'aerialway', 'barrier',
                      'man_made', 'railway'],
    'multipolygons' : ['aeroway', 'amenity', 'barrier', 'building',
                      'landuse', 'leisure', 'man_made', 'military',
                      'natural', 'office', 'waterway', 'power',
                      'railway', 'sport', 'tourism']
}

Qs = [
    " UNION ALL ".join([(
        "SELECT '{keycol}' AS key, {keycol} AS value, "
        "'{geomtype}' AS geom FROM {tbl} WHERE "
        "{keycol} IS NOT NULL"
    ).format(
        keycol=c, geomtype='Point' if table == 'points' else 'Line' if table == 'lines' else 'Polygon',
        tbl=table
    ) for c in TABLES_TAGS[table]]) for table in TABLES_TAGS
]

fileOSMFeatures = sqlq_to_df(sqdb, (
    "SELECT key, value, geom FROM ({tbl}) AS foo "
    "GROUP BY key, value, geom"
).format(tbl=" UNION ALL ".join(Qs)))

In [6]:
print fileOSMFeatures.shape

(327, 3)


In [7]:
print ourOSMFeatures.shape

(450, 3)


In [8]:
_fileOSMFeatures = fileOSMFeatures.merge(
    ourOSMFeatures, how='outer',
    left_on=["key", "value", "geom"],
    right_on=["key_y", "value_y", "geom_y"]
)

print _fileOSMFeatures.shape

(621, 6)


In [14]:
# Select OSM Features of file without correspondence
_fileOSMFeatures["isnew"] =_fileOSMFeatures.key_y.fillna(
    value='nenhum')

newTags = _fileOSMFeatures[_fileOSMFeatures.isnew == 'nenhum']

# Convert to str
def __to_str(row):
    try:
        row[key]   = str(row.key)
        row[value] = str(row.value)
        row['not_conv'] = 0
    except:
        row['not_conv'] = 1
    
    return row

#newTags = newTags.apply(lambda x: __to_str(x), axis=1)

print newTags.shape

newTags["whr"] = newTags.key.str.encode('utf-8').astype(str) + "='" + \
    newTags.value.str.encode('utf-8').astype(str) + "'"

(171, 7)


In [23]:
print newTags.key.str.encode('utf-8').astype(str) + "='" + newTags.value.str.encode('utf-8').astype(str) + "'"

0          aerialway='cable_car'
1          aerialway='drag_lift'
2              aerialway='goods'
3            aeroway='aerodrome'
4               aeroway='hangar'
5              aeroway='helipad'
6               aeroway='runway'
7             aeroway='terminal'
9                  amenity='bar'
13           amenity='childcare'
19      amenity='drinking_water'
30           amenity='monastery'
31            amenity='mortuary'
42     amenity='public_building'
44          amenity='residencia'
47      amenity='slaughterhouse'
54             barrier='bollard'
55               barrier='chain'
56           barrier='city_wall'
57               barrier='fence'
58               barrier='fence'
59          barrier='fence;wall'
60                barrier='gate'
61          barrier='guard_rail'
62               barrier='hedge'
63      barrier='jersey_barrier'
64                barrier='kerb'
65      barrier='retaining_wall'
66                barrier='step'
67                barrier='wall'
          

In [13]:
#display(newTags, raw=True)
import pandas

with pandas.option_context('display.max_rows', None, 'display.max_columns', None):
    print newTags[newTags['not_conv'] == 1]

           key                    value     geom  key_y  value_y  geom_y  \
0    aerialway                cable_car     Line    NaN      NaN     NaN   
1    aerialway                drag_lift     Line    NaN      NaN     NaN   
2    aerialway                    goods     Line    NaN      NaN     NaN   
3      aeroway                aerodrome  Polygon    NaN      NaN     NaN   
4      aeroway                   hangar  Polygon    NaN      NaN     NaN   
5      aeroway                  helipad  Polygon    NaN      NaN     NaN   
6      aeroway                   runway  Polygon    NaN      NaN     NaN   
7      aeroway                 terminal  Polygon    NaN      NaN     NaN   
9      amenity                      bar  Polygon    NaN      NaN     NaN   
13     amenity                childcare  Polygon    NaN      NaN     NaN   
19     amenity           drinking_water  Polygon    NaN      NaN     NaN   
30     amenity                monastery  Polygon    NaN      NaN     NaN   
31     ameni

In [11]:
for t in TABLES_TAGS:
    if t == 'points':
        filterDf = newTags[newTags.geom == 'Point']
    
    elif t == 'lines':
        filterDf = newTags[newTags.geom == 'Line']
    
    elif t == 'multipolygons':
        filterDf = newTags[newTags.geom == 'Polygon']
    
    shp = sel_by_attr(
        sqdb,
        u"SELECT * FROM {} WHERE {}".format(t, 
            filterDf.whr.str.cat(sep=" OR ")
        ),
        os.path.join(WORKSPACE, t + '.shp')
    )

UnicodeEncodeError: 'ascii' codec can't encode character u'\xed' in position 882: ordinal not in range(128)